In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets, linear_model
import cmath 
import csv
import baostock as bs

# fun

In [5]:
def loading_prices(stocks_id_str,start_date,end_date):
        k_data_df = pd.DataFrame()
        rs = bs.query_history_k_data_plus(stocks_id_str,"date,close",start_date=start_date, end_date=end_date, frequency="d", adjustflag="3")
        k_data_df = k_data_df.append(rs.get_data())
        k_data_df.set_index(keys = 'date',inplace=True)
        k_data_df['close'] = k_data_df['close'].astype('float')
        k_data_df.rename(columns={"close": stocks_id_str},inplace=True)
        return k_data_df
    
def load_stocks_prices(stocks_id,start_date,end_date):
    history_df = pd.DataFrame()
    lg = bs.login()
    for i in stocks_id:
        #print(i)
        k_data_df = loading_prices(i,start_date,end_date)
        history_df = pd.concat([history_df,k_data_df],axis=1)
    bs.logout()
    return history_df

# loading and preview data

In [2]:
fundid = 'fund000404'
reprort_date = ['2019-09-30','2019-12-31','2020-03-31','2020-06-30','2020-08-24']

In [219]:
reg_df = pd.read_csv("reg_fund000404_2020-03-31.csv")
reg_df.set_index(keys = 'date',inplace=True)
reg_df.head(2)

,stocks_unit_value,unit_net_value,cum_net_value,daily_return,subscription_status,redemption_status,dividend,sh.000300,hs300_daily_return,stocks_daily_return,dist
date,,,,,,,,,,,
2019-12-31,3.479516e+09,2.924,2.924,0.27,True,True,NaN,4096.582,NaN,NaN,58
2020-01-02,3.568643e+09,3.001,3.001,2.63,True,True,NaN,4152.240,1.358645,2.561478,57


In [215]:
result_df = pd.read_csv('result.csv')
result_df.head(2)

,fundid,date,reg,train_r2,test_r2,train_var,test_var,test_var1,test_var2,ratio,disc,comments,coef,intercept
0,fund000404,2019-12-31,daily_return,0.909691,0.965560,0.054609,0.062405,0.069116,0.102950,1.000000,1.00,n,[0.83006128 0.14928801],0.038142
1,fund000404,2019-12-31,daily_return,0.875264,0.946744,0.064179,0.077602,0.129012,0.087973,0.857533,0.98,c,[1.49270725 0.55458131],0.013085


In [7]:
stocks_details= pd.read_csv(fundid+"_"+reprort_date[i]+".csv")
stocks_details[stocks_details['ipoDate']>'2019-01-01']

,code_name,ratio,num_shares,fund_value,code,ipoDate,outDate,type,status
3,壹网壹创,4.33,1236000.0,280569000.0,sz.300792,2019-09-27,NaN,1,1
10,值得买,2.87,1303000.0,185912300.0,sz.300785,2019-07-15,NaN,1,1
11,新媒股份,2.62,1099700.0,169357200.0,sz.300770,2019-04-19,NaN,1,1


In [240]:
stocks_details.head(2)

,code_name,ratio,num_shares,fund_value,code,ipoDate,outDate,type,status
0,卫宁健康,9.54,29446800.0,617499300.0,sz.300253,2011-08-18,NaN,1,1
1,三花智控,7.51,30242800.0,486002500.0,sz.002050,2005-06-07,NaN,1,1


In [241]:
report_date_fund_value = stocks_details['fund_value'][0]/(stocks_details['ratio'][0]/100)

6472738993.710692

In [249]:
ratio_at_fund  = stocks_details['fund_value']/report_date_fund_value  # ==stocks_details['ratio']
stocks_details['ratio'].sum()/100,ratio_at_fund.sum()

(0.5732, 0.5732828256485472)

In [248]:
ratio_at_pro = ratio_at_fund /ratio_at_fund.sum()

# Input1: return of basic stock 

In [153]:
df = pd.read_csv("hs300_2019-01-01.csv")
df.set_index(keys = 'date',inplace=True)

In [213]:
df.head(4)

,sh.000300
date,
2019-01-02,2969.535
2019-01-03,2964.842
2019-01-04,3035.874
2019-01-07,3054.303


In [190]:
df_return = (df-df.shift(1))/df.shift(1)
hs = df_return ['2020-01-01':'2020-03-31'].values

In [192]:
np.sqrt(hs.var()),np.std(hs)
vol_year_hs= np.sqrt(hs.var()*252)
vol_daily_hs = np.sqrt(hs.var())
vol_year_hs #0.3075949906782444

0.3075949906782444

# input2: return of profolio (breakdown)

In [277]:
#history = load_stocks_prices(stocks_details['code'],stocks_details['ipoDate'].max(),reprort_date[i])
data = history.copy()
logrets = np.log(data / data.shift(1))
logrets = logrets['2020-01-01':'2020-03-31']
rets = (data-data.shift(1))/data.shift(1)
rets = rets['2020-01-01':'2020-03-31']

In [331]:
# breakdown of profolio
weights = ratio_at_pro
vol_yearly_pro =  np.sqrt(np.dot(weights.T, np.dot(rets.cov() * 252, weights)))
print("yearly:",vol_yearly_pro ,"combined")

yearly: 0.4750516777959708 combined


In [264]:
rets.head(2)

,sz.300253,sz.002050,sz.300750,sz.300792,sz.002850,sz.300496,sz.002120,sz.300413,sh.600276,sz.300207,sz.300785,sz.300770,sz.300226,sz.300662
date,,,,,,,,,,,,,,
2020-01-02,0.010681,0.016157,0.010526,0.100017,-0.009670,0.027470,-0.011111,0.100114,0.001828,0.031762,0.099986,0.052692,0.001542,-0.018874
2020-01-03,0.001982,0.023282,0.018880,-0.001750,0.005992,-0.005821,-0.015184,0.023141,-0.019731,-0.015392,-0.021588,-0.001169,0.031828,-0.006733


# Supposed output: vol of fund

In [143]:
fund_value_file = "value000404_2019-09-30.csv"
#_value = pd.read_csv(fund_value_file)
#_value.set_index(keys = 'date',inplace=True)
_value.head()

,unit_net_value,cum_net_value,daily_return,subscription_status,redemption_status,dividend
date,,,,,,
2019-09-30,2.408,2.408,-1.27,True,True,NaN
2019-10-08,2.357,2.357,-2.12,True,True,NaN
2019-10-09,2.381,2.381,1.02,True,True,NaN
2019-10-10,2.428,2.428,1.97,True,True,NaN
2019-10-11,2.406,2.406,-0.91,True,True,NaN


In [148]:
#weight is 
b = _value['daily_return']['2020-01-01':'2020-03-31']/100
np.sqrt(b.var()),np.std(b)

(0.026277385565073082, 0.026049871451044312)

In [196]:
fund_price = _value['cum_net_value']
fund_return = (fund_price-fund_price.shift(1))/fund_price.shift(1)
fund_return  = fund_return ['2020-01-01':'2020-03-31']
print("daily:",np.sqrt(fund_return.var()),np.std(fund_return))
vol_year_fund= np.sqrt(fund_return .var()*252)
vol_daily_fund = np.sqrt(fund_return .var())

daily: 0.026280323708974252 0.026052784155991333


In [197]:
vol_year_fund

0.41718720544932286

# culculation

#### coff read from results.csv

In [ ]:
k1 = 0.7121081
k2 = 0.2538479
kk1 = k1/(k1+k2)
kk2 = k2/(k1+k2)
vol_year_hs #0.3075949906782444
vol_yearly_pro # 0.4750516777959708
vol_yearly_pro_combined #0.4842760072971121

### calculate daily return of new profolio

In [315]:
k1_ratio_at_pro = ratio_at_pro*k1
kk1_ratio_at_pro = ratio_at_pro*kk1

In [303]:
combine_return = pd.concat([rets, df_return ['2020-01-01':'2020-03-31']],axis=1)#df_return from hs300

### supposed output 

In [290]:
vol_year_fund # 0.41718720544932286

0.41718720544932286

### predict output

In [326]:
weights = np.append(kk1_ratio_at_pro,[kk2])
print(weights.sum(),np.sqrt(np.dot(weights.T, np.dot(combine_return.cov() * 252, weights))))

1.0 0.41690171323527553
